In [1]:
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta

geolocator = Nominatim(user_agent="tomte")

# Travel from
fromLoc = geolocator.geocode("Smygehuk")
print(fromLoc.address)
print((fromLoc.longitude, fromLoc.latitude))
# To
toLoc = geolocator.geocode("Åre")
print(toLoc.address)
print((toLoc.longitude, toLoc.latitude))

# Departure
departureTime = datetime.now() # only now supported

Smygehuk, Smygehamn, Trelleborgs kommun, Skåne län, 23178, Sverige
(13.3593569, 55.338238)
Åre, Åre kommun, Jämtlands län, 837 52, Sverige
(13.0822241, 63.4010888)


In [2]:
import openrouteservice
from openrouteservice import convert

# Get route
travel = ((fromLoc.longitude, fromLoc.latitude),(toLoc.longitude, toLoc.latitude))
client = openrouteservice.Client(key='5b3ce3597851110001cf6248e5e0e122139f4ad58f1ca53c410291a6') # Specify your personal API key
routes = client.directions(travel)

steps = routes['routes'][0]['segments'][0]['steps']
route_coords = convert.decode_polyline(routes['routes'][0]['geometry'])['coordinates']

# Shift lat long
for c in route_coords:
    c.append(c.pop(0))

In [3]:
# Find where and when to sample the forecast 
weather_intervall_s = 1*60*60 # Try to sample every hour
places = []
time = []
t = 0
tmp = 0
for idx, step in enumerate(steps):
    tmp = tmp + step['duration']
    t = t + step['duration']
    if idx == 0 or tmp >= weather_intervall_s:
        places.append(route_coords[step['way_points'][1]])
        time.append(t)
        tmp = 0
# Add destinaltion as last place. Replace if close to last place 
if(tmp < weather_intervall_s/2):
    places.pop(-1)
    time.pop(-1)
places.append(route_coords[-1])
time.append(t)


In [7]:
from pysmhi.pysmhi import PySmhi

# Collect forecasts
ps = PySmhi()
r={}
r['coords'] = []
r['forecast'] = []
r['time'] = []
for idx, coord in enumerate(places):
    hours = round(time[idx] / 60 / 60) # When are we there? Hours from now
    w = ps.getWeather(1+idx, coord[0], coord[1])  # Get weather for place and time
    # Store result
    if w:
        r['coords'].append(coord)
        r['forecast'].append(w[-1])
        r['time'].append(time[idx])
        
        location = geolocator.reverse("{:f}, {:f}".format(coord[0], coord[1]))
        print(location.address)        
        print(ps.getFcStr(w[-1]))
        print(" ")

Smygehuk, Smyge Strandväg, Smygehuk, Smygehamn, Trelleborgs kommun, Skåne län, 23178, Sverige
temp: 5.2C wind: 11.8 (15.7)m/s relHumid:  87%, weahter: Light rain
 
E 6;E 20, Axeltofta, Landskrona kommun, Skåne län, 26144, Sverige
temp: 4.3C wind: 7.7 (13.5)m/s relHumid:  92%, weahter: Light rain
 
Trafikplats Hornsborg, E 4, Pettersdal, Ljungby kommun, Kronobergs län, 287 72, Sverige
temp: 2.4C wind: 3.4 (8.4)m/s relHumid:  99%, weahter: Light rain
 
Trafikplats Torsvik, E 4, Moliden, Jönköping, Jönköpings kommun, Jönköpings län, 55652, Sverige
temp: 1.6C wind: 3.5 (7.8)m/s relHumid:  98%, weahter: Light rain
 
Trafikplats Mjölby Östra, E 4, Kungshöga, Habblarp, Mjölby, Mjölby kommun, Östergötlands län, 59534, Sverige
temp: 1.4C wind: 2.2 (4.6)m/s relHumid:  96%, weahter: Overcast
 
Trafikplats Gammelsta, E 4, Bångtorp, Nyköpings kommun, Södermanlands län, 61190, Sverige
temp: 1.2C wind: 2.0 (4.5)m/s relHumid:  98%, weahter: Overcast
 
Trafikplats Hallunda, Södertäljevägen, Hallunda, B

KeyboardInterrupt: 

In [137]:
import folium

# Create map and draw route
from IPython.display import display
m = folium.Map(location=r['coords'][0], zoom_start=6)
folium.PolyLine(route_coords,
                color='red',
                weight=5,
                opacity=0.5).add_to(m)

In [138]:
# Add forecast to map
from folium.features import DivIcon
from suntime import Sun

for idx, place in enumerate(r['coords']):

    timeWhenThere = departureTime + timedelta(seconds=r['time'][idx])

    # Wweather icons
    if r['forecast'][idx][-1] == 'Overcast':
        useIcon = 'cloud'
    else:
        useIcon = 'sun-o'

    # Sun or dark
    sun = Sun(place[0], place[1])
    if sun.get_local_sunrise_time().time() < timeWhenThere.time() < sun.get_local_sunset_time().time():
        iconColor = 'lightblue'
    else:
        iconColor = 'gray'

    # Add marker with icon
    folium.Marker(place, icon=folium.Icon(color=iconColor, icon_color='white', icon=useIcon, prefix='fa')).add_to(m)
    # Add forecast text
    folium.Marker(place, 
            icon=DivIcon(
                icon_size=(450,36),
                icon_anchor=(-25,20),
                html='<div style="font-size: 12pt; color : black">{:s} {:2.1f}°C {:s}</div>'.format(
                    timeWhenThere.astimezone().strftime("     %H:%M"), 
                    r['forecast'][idx][0],
                    r['forecast'][idx][-1]),
                )).add_to(m)
# Display map
m
